In [1]:
from pyserini.index import IndexReader 
from pyserini.search import SimpleSearcher
import gensim
from gensim.models.word2vec import Word2Vec
import gensim.downloader as api
import json
import torch
import numpy as np
import codecs
from bs4 import BeautifulSoup
from tqdm import tqdm
from utils import *

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

config = json.loads(open("config.json", "r").read())
index_path = config["index_path"]
topics_path = config["topics_path"]
qrels_path = config["qrels_path"]
index_path
device = torch.device('cpu')

/home/marco/anaconda3/envs/IR/lib/python3.9/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
# wiki_50 = api.load('glove-wiki-gigaword-50')
wiki_300 = api.load('glove-wiki-gigaword-300')
# google = api.load('word2vec-google-news-300')


In [3]:
expand_query('hubble space telescope', wiki_300, 10)

'hubble space telescope nasa spacecraft observatory astronomers infrared shuttle astronauts nasa observatories orbiting'

# Results

In [30]:
topics = get_topics(topics_path)
#k = nro. results per topic
#n = nro. extra words
k = 400
n = 10

def make_results(model, n:int, dynamic = False, k:int = 25):
    results = ""
    for i in tqdm(topics):
        ranking = ""
        if n > 0:
            if dynamic:
                n = len(topics[i].split())
            expanded_topic = expand_query(topics[i], model, n)
            hits = Searcher.search(expanded_topic, k=k)
        else:
            hits = Searcher.search(topics[i], k=k)
        for r, h in enumerate(hits):
            ranking += f"{i} 0 {h.docid} {r+1} {h.score} RUN1\n"
        results += ranking
        
    filename = ""
    if not dynamic:
        filename = f'results/results_{n}_{k}.txt'
    else:
        filename = f'results/results_x_{k}.txt'
        
    f = open(filename, 'w')
    f.write(results)
    f.close()
    
def create_test_data(model, n:int, k:int = 25):
    make_results(model, 0, True, k) # no expanded query
    make_results(model, n, True, k) # expanded query
    

create_test_data(wiki_300, n, k=k)

100%|██████████| 250/250 [00:42<00:00,  5.89it/s]


In [31]:
metric = 0

labels_gen = query_labels_from_file(qrels_path, f'results/results_x_{k}.txt')
r = 0
for labels in labels_gen:
    metric+=NDCG(labels, k)
    r+=1
     
print(f"{n} {k} {metric/r}")

10 400 0.588354224730254


In [32]:
metric_0 = 0

labels_gen = query_labels_from_file(qrels_path, f'results/results_0_{k}.txt')
r = 0
for labels in labels_gen:
    metric_0+=NDCG(labels, 25)
    r+=1
    
print(f"0 {k} {metric_0/r}")

0 400 0.34524561346018595
